In [19]:
import json
from pathlib import Path
import networkx as nx
import matplotlib.pyplot as plt
from python_fbas.fbas_graph import *

# Using PageRank to identify the top tier

In [25]:
# load the top-tier FBAS:
notebook_dir = Path().resolve()
f = notebook_dir / "data/top_tier.json"
# load file as json:
with open(f, "r") as file:
    data = json.load(file)
fbas = FBASGraph.from_json(data, from_stellarbeat=True)



In [35]:
# try plotly
import plotly.graph_objects as go

G = fbas.graph  # This is a nx.DiGraph
pos = nx.spring_layout(G)  # Node positions

# Extract edge positions
edge_x = []
edge_y = []
edge_annotations = []  # To store arrow annotations
edge_text = []  # Edge hover labels

for edge in G.edges():
    u, v = edge  # Start and end node
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    
    # Store edge for plotting
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)  # None creates line breaks
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

    # Edge hover label
    edge_text.append(f"{u} → {v}")  # Unicode arrow

    # Arrow position (slightly before reaching the destination node)
    arrow_x = x0 + 0.9 * (x1 - x0)  # Adjust position along the edge
    arrow_y = y0 + 0.9 * (y1 - y0)

# Edge trace (lines with hover text)
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='gray'),
    hoverinfo='text',
    text=edge_text,  # Show edge direction on hover
    mode='lines+markers',
    marker=dict(symbol='arrow', size=20, angleref='previous')
)

# Node trace (hover labels)
node_x = []
node_y = []
node_text = []  # Hover labels

for node_id, data in G.nodes(data=True):
    x, y = pos[node_id]
    node_x.append(x)
    node_y.append(y)
    label = f"{data['name'] if 'name' in data else node_id}"
    node_text.append(label)  # Label on hover

# Node trace (scatter points with hover labels)
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    marker=dict(size=10, color='blue'),
    hoverinfo='text',  # Only show labels on hover
    text=node_text  # Labels
)

# Create the figure
fig = go.Figure(data=[edge_trace, node_trace])
fig.update_layout(
    showlegend=False,
    hovermode="closest",  # Enables per-node hover
    plot_bgcolor='white',
    margin=dict(b=20, l=5, r=5, t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    annotations=edge_annotations  # Add arrows
)

# Show the interactive graph
fig.show()
